7 Sept 2023

I idenfified a few formulas that contribute to CO2 equivalent calculation. There are very many steps to the calculation.
The formulas take variable `fuel consumption`. I cannot establish where does this variable come from? This will be my next step

**Issue**
calc_CO2_fuel(fuel_consumption) does not consider distance travelled. This seems like a mistake, it should consider distance



In [ ]:
# The following Conversion factors are needed to CO2-equivalent for a given trip 

CO2 = {
    "FUEL": 3.17231,          # * FC * DIST
    "LUBE": 0.265170857776354,  # * DIST
    "ELECTRIC": 119           # * DIST
}

CH4 = {
    "PRE-EURO": 0.175,  # * DIST
    "EURO1": 0.175,
    "EURO2": 0.11375,
    "EURO3": 0.10325,
    "EURO4": 0.00525,
    "EURO5": 0.00525,
    "EURO6": 0.00525,
    "CO2_FACTOR": 25
}

N2O = {
    "PRE-EURO": 0.030,  # * DIST
    "EURO1": 0.012,
    "EURO2": 0.012,
    "EURO3": 0.006,
    "EURO4": 0.0128,
    "EURO5": 0.0332,
    "EURO6": 0.0415,
    "CO2_FACTOR": 298
}

DIESEL_DENSITY = 835  # grams per litre

# These functions calculate CO2 emissions components and feed into the calc_CO2_equiv function:

def calc_CO2_fuel(fuel_consumption):
    return fuel_consumption * DIESEL_DENSITY * CO2['FUEL']

def calc_CO2_oil(distance):
    return distance * CO2['LUBE']

def calc_N2O(distance, euro):
    return distance * N2O[euro]

def calc_CH4(distance, euro):
    return distance * CH4[euro]

# This function calculates CO2 equivalent
# We need to do a lot of untangling to bring all the subfucntions and constants into it

# Calculate total CO2-equivalent for a given route
# fuel in litres
# distance in kilometres

def calc_CO2_equiv(fuel_consumption, distance, euro_rating):
    if euro_rating == 'EV':
        return (distance * CO2['ELECTRIC'] / 1000)
    fuelEmissions = calc_CO2_fuel(fuel)
    oilEmissions = calc_CO2_oil(distance)
    N2OEmissions = calc_N2O(distance, euro_rating) * N2O['CO2_FACTOR']
    CH4Emissions = calc_CH4(distance, euro_rating) * CH4['CO2_FACTOR']
    totalKgCO2 = (fuelEmissions + oilEmissions + N2OEmissions + CH4Emissions) / 1000
    return totalKgCO2

# Description of calculation from Josh:

The formulas you have calculate the emissions in grams per KM. 
**Electric bus**

Use different formula

**Step 1.** Calculate weight factors:

PASSENGERWEIGHT = 80

loadedWeight = tareWeight + aveOccupancy * PASSENGERWEIGHT

tareFactor =   (0.00004711 * tareWeight) + 0.446;

loadedFactor = (0.00004711 * loadedWeight) + 0.446;


**Step 2.** Calculate the emissions per KM (g):

(a*speed^2 + b*speed + g + d/speed) / (e*speed^2 + z*speed + h)

To get a,b, g, d, e, z and h see file front-end/src/calculations/pollutants.tsx, line 11

**Step 3.** Only applicable to CO2 emissions.

CO2 (g) = fuel emissions + oil emissions  

fuel emissions (g) = Fuel Consumed (L) * 69.31 * 835 * 45.79/100000

oil emissions (g) = distance * LUBE

**note** (LUBE = 0.265170857776354)

CO2 (g) = Fuel Consumed (L) * 69.31 * 835 * 45.79/100000 + distance * LUBE

let x = speed (km/h)

Fuel Consumed for standard bus (L/km) = 100 / (45.79 * 835/1000) * (a * speed^2+b * speed+g+d/speed) / (e*speed^2 + z*speed + h)

Fuel Consumed for our bus (L) = Fuel Consumed for standard bus (L/km) * (loaded weightFactor * distanceKM + empty WeightFactor * distanceKM*0.15) 

**Note** Empty WeightFactor * distanceKM*0.15 is repositioning distance

The 0.15 is accounting for an approximation 15% of the journey the bus travels to either get back to the bus depo or start the next route


CO2 (g) = (100 / (45.79 * 835/1000) * (a * speed^2+b * speed + g + d/speed))/ (e*speed^2 + z*speed + h) * (loadedFactor * distanceKM + tareFactor * distanceKM * 0.15)* 69.31 * 835 * 45.79/100000 + distance * LUBE 


**Step 4.** Add emissions from all pllutants:

CO2 (from step 3) + N2OEmissions + CH4Emissions



N2OEmissions = distance * N2O[euro]   * N2O.CO2FACTOR;

N2O.CO2FACTOR = 298
N2O.CO2FACTOR converts the N20 realsed to its C02 equivilent. This is to say, how much greenhouse effect does this have over its time in the apnosphere. How much CO2 is needed to have the same effect as this much N20. The numbers we have in the code are used by a previous group however I did my own research and found slightly different numbers. I really want to put them in but I don't know where their numbers come from and I can't find any through research.

CH4Emissions = distance * CH4[euro] * CH4.CO2FACTOR;

CH4.CO2FACTOR = 25

**question** what is N2O[euro] and CH4[euro]?  Do I fund respective figure for example for CH4 here:

 - EURO1"     : 0.175,
 -   "EURO2"     : 0.11375,
 -   "EURO3"     : 0.10325,
 -   "EURO4"     : 0.00525,
 -   "EURO5"     : 0.00525,
 -   "EURO6"     : 0.00525,
 
 N2O and CH4 return kgs rather tham grams, right?
